# This is perhaps only for LDA model because for LDA-GR and latent theta role we will need the original text.. at some point maybe we call this pre-processed data for them if the sample is too large.

# Define extra stop words

In [1]:
#read all the stop words and add them to the list of extra stop words..
extra_stop_words = open('Stopword_list', 'r')
#addiong some stop words...
extra_stop_words_list = extra_stop_words.readlines()
extra_stopwords = set()
for item in extra_stop_words_list:
    extra_stopwords.add(item.strip())
extra_stopwords.add('amp');extra_stopwords.add('&amp');extra_stopwords.add('&amp;')

# print(extra_stopwords)

In [2]:
import pickle
with open('wiki_vocab_dict','rb') as read_file:
    wiki_vocab_dict = pickle.load(read_file)

with open('vocab_dict', 'rb') as read_file:
    vocab_dict = pickle.load(read_file)

/home/zha219/.local/lib/python3.8/site-packages/scipy/__init__.py:143: UserWarning: A NumPy version >=1.19.5 and <1.27.0 is required for this version of SciPy (detected version 1.19.4)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
# set(vocab_dict.token2id.keys())
extra_stopwords = extra_stopwords.union(set(vocab_dict.token2id.keys()).difference(set(wiki_vocab_dict.token2id.keys())))


In [4]:
import pickle

with open('extra_stopwords','wb') as save_file:
    pickle.dump(extra_stopwords,save_file)

# pre-process the documents

In [6]:
#pre-processing 
#this part uses Amin's code 
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim import similarities

import csv, sys
import re
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')

from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet


import pandas as pd
import numpy as np

import copy


try:
	csv.field_size_limit(sys.maxsize)
except:
	print('Error in setting maxSize for CSV output')
    
def preprocess_data(doc_list, id_list, extra_stopwords = {},len_th=4,lemmatized=False):
	'''
	Returns: a list of process dataset and origianl documents of those documents

	This function removes stop-wrods, lemmatized the documens, if stated, and eliminates the documnets 
	with lenhgth of 4 or less. 
	***These processes may result in lower number of documents than the original number. To make sure 
	you receive both the original docs and the processed doc in similar order we return both.

	parameter doc_list: a list of string (documents)
	parameter extra_stopwords: NLTK.stop_words are used, if you wish to add to that list, you can use yours.
	parameter len_th: documents with len_th and less will be removed.
	parameter lemmatized: If true, the terms will be lemmatized. **be aware that lemmatization of the documents
	will result in different topics and may need different evaluation, including NPMI, stability, or human assessment**

	'''

	# replace single smart quote with single straight quote, so as to catch stopword contractions
	doc_list = [re.sub("[\u2018\u2019]", "'", doc) for doc in doc_list] #replace qoute with regualar qoutations
	doc_list = [re.sub('\d+', '', doc) for doc in doc_list]
	doc_list = [re.sub('(\/.*?\.[\w:]+)', '', doc) for doc in doc_list]
	#doc_list = [re.sub('pdf|icon|jpg', '', doc) for doc in doc_list]
	#doc_list = [re.sub('(http(s)?:\/\/.)?(www\.)?[-a-zA-Z0-9@:%._\+~#=]{2,256}\.[a-z]{2,6}\b([-a-zA-Z0-9@:%_\+.~#?&//=]*)', '', doc) for doc in doc_list]
	doc_list = [re.sub(r"http\S+", '', doc) for doc in doc_list]

	# initialize regex tokenizer
	tokenizer = RegexpTokenizer(r'\w+')
	# create English stop words list
	en_stop = set(stopwords.words('english'))
	# add any extra stopwords
	if (len(extra_stopwords) > 0):
		en_stop = en_stop.union(extra_stopwords)

	#defining a lemmatizer
	lemmatizer = WordNetLemmatizer()

	# list for tokenized documents in loop
	texts = []
	original_docs = []
	# loop through document list
	c = 0 #counter on the document number
	for i in doc_list:
		# clean and tokenize document string
		raw = i.lower()
		tokens = tokenizer.tokenize(raw)
		stopped_tokens = []
		# remove stop words from tokens
		#stopped_tokens = [i for i in tokens if not i in en_stop and len(i)>1]
		if lemmatized:
		  for t in tokens:
		    if t not in en_stop and len(t)>1:
		      pos=nltk_tag_to_wordnet_tag(nltk.pos_tag([t])[0][1])
		      if pos:
		        stopped_tokens.append(lemmatizer.lemmatize(t,pos=pos))
		      else:
		        stopped_tokens.append(lemmatizer.lemmatize(t))
		  #     print(t,pos,nltk.pos_tag([t])[0][1])
		  # print(stopped_tokens)
		  #stopped_tokens = [lemmatizer.lemmatize(i,pos=nltk_tag_to_wordnet_tag(nltk.pos_tag([i])[0][1])) for i in tokens if not i in en_stop and len(i)>1]
		else:
		  stopped_tokens = [i for i in tokens if not i in en_stop and len(i)>1]


		# add tokens to list
		if len(stopped_tokens) >= len_th:
		  texts.append([stopped_tokens, i, id_list[c]]) # [pre-processed text (tokenized), original, id]
		  #original_docs.append([i,c])

		c += 1

	return texts

[nltk_data] Downloading package stopwords to /home/zha219/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/zha219/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/zha219/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/zha219/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


# Get the data from DB

In [8]:
import mysql.connector

sql_db = mysql.connector.connect(host='127.0.0.1',user = 'covidAnalysis',password = 'k34p63MbDDcZ9yf4',
                                 database = 'covid19framing')
cursor = sql_db.cursor()

InterfaceError: 2013: Lost connection to MySQL server during query

In [ ]:
cursor.execute('show tables;')
results = cursor.fetchall()
results

In [ ]:
cursor.execute('desc twitter')
#cursor.execute('desc twitter_hashtag')
# cursor.execute('desc articles')
results = cursor.fetchall()
results

# Fetch data and store back to the databse the pre-processed data


In [ ]:
cursorUpdate = sql_db.cursor()
offset = 0#counter for offset
results = [0] #starting with a non-empty results to make sure we get into loop in the first place

while results:
    cursor.execute("select * from twitter where text_preproc is NULL limit 1000 offset {0};".format(offset))
    #cursor.execute("select * from articles where text_preproc is NULL limit 1000 offset {0};".format(offset))
    offset += 1000
    results = cursor.fetchall()
    if not results:  # Check if there are no more results
        break
    result_dic = {i[0]:i for i in results}
#     print(type([i[0] for i in results]), type([i[1].decode() for i in results]))
    
#     print(results[0])1
    #send to pre_process (...)
    out = preprocess_data([i[3].decode() for i in results], [i[0] for i in results], extra_stopwords,len_th=4,lemmatized=False)
    #out ==> 1. pre_prcesed doc, original, id
#     print(type(out))
    #print(out)
    #putting it back to DB
    for o in out:
        cursorUpdate.execute("update twitter SET text_preproc = '{0}' where id = {1}".format(' '.join(o[0]),o[2]))
        #print(o[0], o[2])
    #cursor.fetchall()
    #offset += 1000


In [ ]:
# results[0][-1].decode().split()

In [ ]:
# o[0], o[1], o[2]
for o in out:
    cursor.execute("update twitter SET text_preproc = '{0}' where id = {1}".format(' '.join(o[0]),o[2]))

In [ ]:
for o in out:
    cursor.execute("update twitter SET text_preproc = '{0}' where id = {1}".format(' '.join(o[0]),o[2]))
    print(o[0], o[2])
    break

In [ ]:
cursor.execute("select * from twitter where text_preproc is not NULL;")
res =  cursor.fetchall()
len(res)

# Commit to save the changes in the DB

In [ ]:
 
sql_db.commit()